In [1]:
import warnings
warnings.filterwarnings("ignore")

In [41]:
import re
import nltk
import numpy as np
import pandas as pd
import sys
import os
import json
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn import preprocessing

df = pd.read_csv('Full_English.csv').drop(columns='Unnamed: 0', axis=1)
df.head()
nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

def max_length(tensor):
    return max(len(t) for t in tensor)

def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
        
    return padded    

class ConstructVocab():
    
    def __init__(self, sentences):
        self.sentences = sentences
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        self.create_index()
        
    def create_index(self):
        for sent in self.sentences:
            self.vocab.update(sent.split(' '))
        
        #sort vacabulary
        self.vocab = sorted(self.vocab)
        
        #add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        #word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # 0 is the pad
            
        #index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word



In [51]:
#df['token_size'] = df['Verse'].apply(lambda x: len(x.split(' ')))
#df['VerseRe'] = df['Verse'].apply(lambda x: re.sub(r'(\w)(\1{2,})', r'\1',x))
df['Verse_norm'] = df['Verse'].apply(normalize_document)
df['token_size_norm'] = df['Verse_norm'].apply(lambda x: len(x.split(' ')))

df = df.loc[df['token_size_norm'] < 60]
df = df.sample(n= 50000)

In [52]:
from sklearn.preprocessing import LabelEncoder
lbl_encode = LabelEncoder()
df['MeterLabels'] = lbl_encode.fit_transform(df['Meter'])

In [53]:
PyT = df.drop(['Verse',	'Meter'	,'char_count'], axis=1)

In [54]:
PyT

Verse_norm  token_size_norm  MeterLabels
126041               sir im glostershire divine                4            2
57601            fearless foremost ranks fought                4            2
75709   darker prospect scarce one gleam behind                6            2
46891                     pride lust charge lay                4            2
47664            picturd regions imagind worlds                4            2
...                                         ...              ...          ...
18681                    one amongst many known                4            2
68971           cressys palm every wreath grows                5            3
166022      less ostentatious yet studded thick                5            2
42081                        drink nectar tears                3            2
41291                 loads learned lumber head                4            2

[50000 rows x 3 columns]

In [55]:
inputs = ConstructVocab(PyT['Verse_norm'].values.tolist())

input_tensor = [[inputs.word2idx[s] for s in es.split(' ')] for es in PyT['Verse_norm']]

max_length_input = max_length(input_tensor)

input_tensor = [pad_sequences(x, max_length_input) for x in input_tensor]

lbls = list(set(PyT.MeterLabels.unique()))

num_lbls = len(lbls)

mlb = preprocessing.MultiLabelBinarizer()

data_labels = [set(rat) & set(lbls) for rat in df[['MeterLabels']].values]

bin_rates = mlb.fit_transform(data_labels)

target_tensor = np.array(bin_rates.tolist())

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=1000)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=1000)

In [57]:
embedding_dim = 256
units = 1024
vocab_inp_size = len(inputs.word2idx)
target_size = len(target_tensor[0])

In [58]:
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        self.length = [np.sum(1 - np.equal(x,0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        
        return x, y, x_len
    
    def __len__(self):
        return len(self.data)

In [59]:

TRAIN_BUFFER_SIZE = len(X_train)
VAL_BUFFER_SIZE = len(X_val)
TEST_BUFFER_SIZE = len(X_test)
BATCH_SIZE = 64

TRAIN_N_BATCH = TRAIN_BUFFER_SIZE // BATCH_SIZE
VAL_N_BATCH = VAL_BUFFER_SIZE // BATCH_SIZE
TEST_N_BATCH = TEST_BUFFER_SIZE // BATCH_SIZE
train_dataset = MyData(X_train, y_train)
val_dataset = MyData(X_val, y_val)
test_dataset = MyData(X_test, y_test)

train_dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE,
                          drop_last=True, shuffle=True)
val_dataset = DataLoader(val_dataset, batch_size = BATCH_SIZE,
                          drop_last=True, shuffle=True)
test_dataset = DataLoader(test_dataset, batch_size = BATCH_SIZE,
                          drop_last=True, shuffle=True)

In [82]:
import torch.nn as nn

class RateGRU(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_units, batch_sz, output_size):
        super(RateGRU, self).__init__()
        self.batch = batch_sz
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_units = hidden_units
        self.output_size = output_size
        
        #layers
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.gru = nn.GRU(self.embedding_dim, self.hidden_units)
        self.fc = nn.Linear(self.hidden_units, self.output_size)
        
    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch, self.hidden_units)).to(device)
    
    def forward(self, x, lens, device):
        x = self.embedding(x)
        self.hidden = self.initialize_hidden_state(device)
        output, self.hidden = self.gru(x, self.hidden)
        out = output[-1, :, :]
        out = self.dropout(out)
        out = self.fc(out)
        
        return out, self.hidden

In [83]:
use_cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = RateGRU(vocab_inp_size, embedding_dim, units, BATCH_SIZE, target_size)
model.to(device)

#loss criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

def loss_function(y, prediction):
    target = torch.max(y, 1)[1] 
    loss = criterion(prediction, target)
    
    return loss

def accuracy(target, logit):
    target = torch.max(target, 1)[1]
    corrects = (torch.max(logit, 1)[1].data == target).sum()
    accuracy = 100. * corrects / len(logit)
    
    return accuracy

In [ ]:
EPOCHS = 10
import time
data_ret = {'predictions':[], 'loss':[], 'batch_loss':[], 'total_loss':[],
            'batch_accuracy':[], 'train_accuracy':[],'val_accuracy':[],}
for epoch in range(EPOCHS):
    
    start = time.time()
    total_loss = 0
    train_accuracy, val_accuracy = 0, 0
    
    for (batch, (inp, targ, lens)) in enumerate(train_dataset):
        loss = 0
        predictions, _ = model(inp.permute(1, 0).to(device), lens, device)
        #print(_)
        data_ret['predictions'].append(_)
        
        loss += loss_function(targ.to(device), predictions)

        data_ret['loss'].append(loss)

        batch_loss = (loss / int(targ.shape[1]))
        data_ret['batch_loss'].append(batch_loss)

        total_loss += batch_loss
        data_ret['total_loss'].append(total_loss)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_accuracy = accuracy(targ.to(device), predictions)
        data_ret['batch_accuracy'].append(batch_accuracy)

        train_accuracy += batch_accuracy
        data_ret['train_accuracy'].append(train_accuracy)



In [85]:

#pip install torchviz

In [97]:
#pip install torchplot

In [98]:
import torchplot as plt

In [110]:
data_ret.keys

dict_keys(['predictions', 'loss', 'batch_loss', 'total_loss', 'batch_accuracy', 'train_accuracy', 'val_accuracy'])

In [124]:
plt.scatter(data_ret['batch_accuracy'], data_ret['train_accuracy']) 

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 <a list of 10 Patch objects>)

In [33]:

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.Verse)
X_train_counts.shape

X_test_counts = count_vect.transform(X_test.Verse)

count_vect.vocabulary_.get(u'algorithm')

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split



X_trainCls, X_testCls, y_trainCls, y_testCls = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=1000, shuffle=target_tensor)
#X_valCls, X_testCls, y_valCls, y_testCls = train_test_split(X_valCls, y_valCls, test_size=0.5, random_state=1000)



In [38]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# With out pipeline
# Define the classifier
clf = RandomForestRegressor()

# Train the model on the training data
clf.fit(X_trainCls, y_trainCls)

# Make predictions on the test data
y_pred = clf.predict(X_testCls)

# Evaluate the model's performance
accuracy = clf.score(X_testCls, y_pred)
print('Accuracy of the model on the test set: {:.2f}%'.format(accuracy * 100))


Accuracy of the model on the test set: 100.00%
